In [ ]:
import uproot as ur
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak


## Instructions:
### 1) Choose angle and particle in cell below
### 2) Set Angle range
### 3) Set weighting range, draw the histograms without fitting
### 4) Ignore the histograms that aren't Gaussian, fit those that are. Feel free to change weighting range.


### Get data, transform ROOT tree into array

In [ ]:
# Parameters 
energy = 1 # Energy in GeV. Available energies are 1, 2, 5, 10, 20, 50 GeV.
particle = 'pi+' # Particle incident on detectors

#files = [] # List to contain all the data file names

# Loop through some large number to obtain all the available files so we don't have to look them up manually. 
# for file_number in range(1, 101):
#     try:
#         file_name = 'root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/{particle}\
# /{energy}GeV/3to50deg/{particle}_{energy}GeV_3to50deg.{file_number:04d}.root:events'.format(energy = energy, particle = particle, file_number = file_number)
#         ur.open(file_name) # Open each file. If the file doesn't exist, it will be skipped and not be added to files list.
#         files.append(file_name)
#     except:
#         continue

#print("Number of files: {NumFiles}.".format(NumFiles = len(files)))

# Files on Xrootd server aren't updated yet so use locally downloaded files
files = '/home/ryan/EIC_Data/{energy}GeV/*.root:events'.format(energy = energy)


# Combining all the data files that we found. Only include the branches that are of interest
full_events = ur.concatenate(files, ["mcparticles2.psx", "mcparticles2.psy", "mcparticles2.psz", "mcparticles2.pdgID"], library = 'np')
EcalP_events = ur.concatenate(files, ["EcalEndcapPHitsReco.energy"], library = 'np')
HcalHadron_events = ur.concatenate(files, ["HcalHadronEndcapHitsReco.energy"], library = 'np')


In [ ]:
if len(full_events['mcparticles2.pdgID']) != len(HcalHadron_events["HcalHadronEndcapHitsReco.energy"]) or len(full_events['mcparticles2.pdgID']) != len(EcalP_events["EcalEndcapPHitsReco.energy"]):
    raise ValueError("Number of events do not align!")
num_events = len(full_events['mcparticles2.pdgID'])
print('Number of events: {events}'.format(events=num_events))

In [ ]:
# Getting an array of angles for each event
theta_array = []
for i in range(num_events):
    if len(full_events["mcparticles2.pdgID"][i]) == 0:
        theta_array.append(-999)
        continue
    if full_events["mcparticles2.pdgID"][i][0] != 211:
        print(i)
    px = full_events["mcparticles2.psx"][i][0]
    py = full_events["mcparticles2.psy"][i][0]
    pz = full_events["mcparticles2.psz"][i][0]
    p = np.sqrt(px*px + py*py + pz*pz)
    theta = np.arccos(pz/p)*180/np.pi
    theta_array.append(theta)

theta_data, theta_bins, _ = plt.hist(theta_array, 47, (3, 50), density=False, facecolor='b', alpha=.5)
plt.title(r"$\bf{ATHENA:}$"+' $\pi^+$ {energy} GeV'.format(energy=energy, fontsize=18))
plt.xlabel("Angle (degrees)", fontsize = 16)
plt.ylabel("Number of Events", fontsize = 16)

In [ ]:
def Angle_Energy(detector_events = HcalHadron_events, detector_name = "HcalHadronEndcapHitsReco", event_angles = theta_array, desired_low_angle = 5, desired_high_angle = 10):
    num_events = len(theta_array)
    energies = []
    for i in range(num_events):
        if event_angles[i] >= desired_low_angle and event_angles[i] < desired_high_angle:
            energies.append(detector_events["{detector}.energy".format(detector = detector_name)][i])
    return energies

### Choosing Angle Range

In [ ]:
# If you want a specific angle range, set filter_angle to True and input angle range into angle variables. Will select events with angles in range [low, high)
filter_angle = True
low_angle = 3
high_angle = 10

Hcal_energies = []
Ecal_energies = []
if filter_angle:
    Hcal_energies = Angle_Energy(HcalHadron_events, "HcalHadronEndcapHitsReco", theta_array, low_angle, high_angle)
    Ecal_energies = Angle_Energy(EcalP_events, "EcalEndcapPHitsReco", theta_array, low_angle, high_angle)
else:
    Hcal_energies = HcalHadron_events["HcalHadronEndcapHitsReco.energy"]
    Ecal_energies = EcalP_events["EcalEndcapPHitsReco.energy"]

num_desired_events = len(Ecal_energies)
print("Considering {X} events".format(X = num_desired_events))

### Setting up weight histograms

In [ ]:
# Weighting the ecal energies and adding to histograms
w0 = 1.5

weights = np.array([],float)
for i in range(100):
    weights = np.append(weights, w0+0.1*i)

EW = [] # EW will hold the weighted total energies for each weight in "weights"
for i in range(len(weights)):
    EW.append([])
    
for i in range(num_desired_events):
    hcal_edep = np.sum(Hcal_energies[i])*1000
    ecal_edep = np.sum(Ecal_energies[i])*1000
    for j in range(len(weights)):
        weighted_energy = ecal_edep/weights[j]+hcal_edep
        EW[j].append(weighted_energy)

### Fitting and choosing optimal weight
#### Set "fit" to true to fit, set "draw" to true to draw all histograms from above cell (Warning: Drawing takes a little bit of time)

In [ ]:
from scipy.stats import norm
from scipy.optimize import curve_fit

def gaussian(x, A, mean, variance):
    return A * np.exp(-0.5*((x - mean)**2 /variance)) # Make sure to take sqrt of variance parameter to get sigma

max_energies = {1:300.5, 2:300.5, 5:400.5, 10:400.5, 20:1500.5, 50:2000.5}

min_resolution = 999999
min_index = -9999
num_bins = 600

max_rows = 10
max_cols = 10

fit = True
draw = False

ignore_hists = 0
if ignore_hists != 0:
    print("Ignoring first {X} histograms.".format(X = ignore_hists))

if draw:
    fig = plt.figure(figsize = (40,40))
    fig.subplots_adjust(hspace = .5)


for i in range(ignore_hists,len(EW)):
    
    if draw:
        plt.subplot(max_rows, max_cols, i+1)   
        hist_data, bins, _ = plt.hist(EW[i], num_bins, (0.5, max_energies[energy]+.5))
        plt.title("Weight: {weight_val}".format(weight_val = round(weights[i],2)))
        plt.xlabel("Energy (MeV)")
        plt.ylabel("Counts")
    else:
        hist_data, bins = np.histogram(EW[i], num_bins, (0.5, max_energies[energy]+.5))

    bin_centers = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])

    if fit:
        try:
            popt, pcov = curve_fit(gaussian, xdata=bin_centers, ydata=hist_data, p0=(100,15,20))
        except:
            continue
        resolution = np.sqrt(popt[2])/popt[1]
        if resolution < min_resolution:
           min_resolution = np.abs(resolution)
           min_index = i
        if draw:
            plt.plot(bin_centers, gaussian(bin_centers, *popt))
            max_bin_height = 0
            for j in range(len(hist_data)):
               if hist_data[j] > max_bin_height:
                   max_bin_height = hist_data[j]
            plt.text(0.8*max_energies[energy]+.5, 0.6*max_bin_height, "$\mu$ = {mean} \n $\sigma$ = {sigma} \n $\sigma/\mu$ = {res}"\
                     .format( mean = round(popt[1],2), sigma = round(np.sqrt(popt[2]),3), res = round(resolution,4)),horizontalalignment='center',multialignment='left',verticalalignment='center', fontsize = 16)



if fit:
    print("Index in array: {index}".format(index = min_index))
    print("Minimum resolution is {res} using weight {weight}.".format(res = round(min_resolution,3), weight = round(weights[min_index],2)))


In [ ]:
# Plotting the histogram with the optimal weight

hist_data, bins, _ = plt.hist(EW[min_index], num_bins, (0.5, max_energies[energy]+.5), density=False, facecolor='b', alpha=.5)
bin_centers = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])
popt, pcov = curve_fit(gaussian, xdata=bin_centers, ydata=hist_data, p0=(100,15,20))
max_bin_height = 0
for i in range(len(hist_data)):
    if hist_data[i] > max_bin_height:
        max_bin_height = hist_data[i]

plt.plot(bin_centers, gaussian(bin_centers, *popt))
plt.xlabel("Edep (MeV)", fontsize = 16)
plt.ylabel("Num. of events", fontsize = 16)
plt.text(0.75*max_energies[energy]+.5, 0.7*max_bin_height, "A = {amplitude} \n $\mu$ = {mean} \n $\sigma$ = {sigma} \n $\sigma/\mu$ = {res}".format(amplitude = round(popt[0],2), mean = round(popt[1],2), sigma = round(np.sqrt(popt[2]),3), res = round(np.sqrt(popt[2])/popt[1],4)),horizontalalignment='center',multialignment='left',
                 verticalalignment='center', fontsize = 16)

plt.title(r"$\bf{ATHENA:}$"+' $\pi^+$ {energy} GeV, ${low_angle}^{degrees}-{high_angle}^{degrees}$'.format(energy=energy, low_angle = low_angle, high_angle = high_angle, degrees = "\circ"), fontsize=18)

### Next steps:
### ~1) Filter the angles. Choose a certain angle, then fill arrays of edep for that angle~
### ~2) Fill a weighting histogram and fit it with a Gaussian to obtain the optimal weight~
### ~3) Fill a new histogram with the optimally weighted energy dependence and fit it to obtain the energy resolution~
### 4) Do this for each energy and then plot (like EIC_EndCap.C)
